In [ ]:
import utilities

In [ ]:
import os
import shutil

import os
import zipfile
import urllib.request

In [ ]:
datasets_dir = os.environ['DOMINO_DATASETS_DIR']
project_ds_folder = os.environ['DOMINO_PROJECT_NAME'] 

download_base_folder=f"{datasets_dir}/{project_ds_folder}"
models_folder = "models"
yolo_model_name="yolov8n"

yolo_onnx_file_name=f"{yolo_model_name}.onnx"

In [ ]:
import os
import shutil
import onnx
import mlflow
import mlflow.onnx
from ultralytics import YOLO
from mlflow.tracking import MlflowClient
from mlflow.store.artifact.runs_artifact_repo import RunsArtifactRepository
from mlflow.exceptions import MlflowException

def download_and_register_yolov_models_with_client(
    download_path: str,
    models_folder: str,
    experiment_name: str = "YOLO_Export"
):

    mlflow.set_experiment(experiment_name)
    client = MlflowClient()

    output_dir = os.path.join(download_path, models_folder)
    os.makedirs(output_dir, exist_ok=True)

    # Models to export -> register names (clean, no extension)
    requests = [
        ("yolov8n.pt", "yolov8n"),
        ("yolov5n.pt", "yolov5n"),
        ("yolov8m.pt", "yolov8m"),
        ("yolov8s.pt", "yolov8s"),
    ]

    registered_versions = {}

    for requested_name, registry_name in requests:
        print(f"[Export] {requested_name} -> ONNX")
        model = YOLO(requested_name)
        onnx_src_path = model.export(format="onnx")  # returns str path to .onnx

        if not onnx_src_path or not os.path.isfile(onnx_src_path):
            raise RuntimeError(f"Export failed for {requested_name}")

        onnx_filename = os.path.basename(onnx_src_path)
        final_path = os.path.join(output_dir, onnx_filename)
        try:
            shutil.move(onnx_src_path, final_path)
        except OSError:
            shutil.copy2(onnx_src_path, final_path)
            os.remove(onnx_src_path)

        if os.path.exists(requested_name):
            os.remove(requested_name)

        # Log ONNX with model flavor so the registry can version it
        print(f"[Log] {registry_name} to MLflow run")
        onnx_model = onnx.load(final_path)

        with mlflow.start_run(run_name=f"register_{registry_name}") as run:
            mlflow.onnx.log_model(onnx_model=onnx_model, artifact_path="model")

            # Build a runs:// URI then convert to the underlying store URI
            runs_uri = f"runs:/{run.info.run_id}/model"
            model_src = RunsArtifactRepository.get_underlying_uri(runs_uri)

            # Ensure Registered Model exists
            try:
                client.create_registered_model(registry_name)
            except MlflowException:
                # Already exists or registry backend returns conflict; proceed to versioning
                pass

            # Create a Model Version pointing to this run’s logged model
            print(f"[Register] {registry_name} -> new version from run {run.info.run_id}")
            mv = client.create_model_version(
                name=registry_name,
                source=model_src,
                run_id=run.info.run_id,
                description=f"Auto-registered {registry_name} ONNX export",
            )
            registered_versions[registry_name] = mv.version

    print("[Done] All models exported and registered.")
    return registered_versions


def load_registered_yolo_model(model_name: str, version: str = "latest"):
    """
    Downloads the registered ONNX model version and returns a Ultralytics YOLO instance.
    """
    import mlflow
    from ultralytics import YOLO

    if version == "latest":
        model_uri = f"models:/{model_name}/latest"
    else:
        model_uri = f"models:/{model_name}/{version}"

    local_dir = mlflow.artifacts.download_artifacts(model_uri)
    # look for single ONNX inside logged model dir
    onnx_candidates = []
    for root, _, files in os.walk(local_dir):
        for f in files:
            if f.endswith(".onnx"):
                onnx_candidates.append(os.path.join(root, f))
    if not onnx_candidates:
        raise FileNotFoundError(f"No .onnx found under {local_dir}")

    onnx_path = onnx_candidates[0]
    return YOLO(onnx_path, task="detect")


In [ ]:
utilities.ensure_mlflow_experiment(utilities.model_registration_experiment_name)
download_and_register_yolov_models_with_client(download_base_folder,models_folder,utilities.model_registration_experiment_name)
#download_faster_rcnn(output_path)

In [ ]:
import os, urllib.request, zipfile, shutil
from pycocotools.coco import COCO

def download_coco_2017_val_ds(base_dir):
    os.makedirs(base_dir, exist_ok=True)

    img_set = "val2017"
    images_url = f"http://images.cocodataset.org/zips/{img_set}.zip"
    ann_url    = "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"

    images_zip = os.path.join(base_dir, f"{img_set}.zip")
    ann_zip    = os.path.join(base_dir, "annotations_trainval2017.zip")

    # download
    if not os.path.exists(images_zip):
        print(f"Downloading {os.path.basename(images_zip)}...")
        urllib.request.urlretrieve(images_url, images_zip)

    if not os.path.exists(ann_zip):
        print(f"Downloading {os.path.basename(ann_zip)}...")
        urllib.request.urlretrieve(ann_url, ann_zip)

    # extract images
    target_images_dir = os.path.join(base_dir, "images", img_set)
    if not os.path.exists(target_images_dir):
        print(f"Extracting {os.path.basename(images_zip)}...")
        with zipfile.ZipFile(images_zip, "r") as zf:
            zf.extractall(base_dir)  # creates base_dir/val2017
        os.makedirs(os.path.join(base_dir, "images"), exist_ok=True)
        src = os.path.join(base_dir, img_set)  # base_dir/val2017
        if os.path.exists(src):
            shutil.move(src, target_images_dir)
        else:
            # already extracted to correct place by prior run
            os.makedirs(target_images_dir, exist_ok=True)

    # extract annotations
    ann_dir = os.path.join(base_dir, "annotations")
    ann_json = os.path.join(ann_dir, f"instances_{img_set}.json")
    if not os.path.exists(ann_json):
        print(f"Extracting {os.path.basename(ann_zip)}...")
        with zipfile.ZipFile(ann_zip, "r") as zf:
            zf.extractall(base_dir)  # creates base_dir/annotations/*.json
    assert os.path.exists(ann_json), f"Missing {ann_json}"

    # build labels
    labels_dir = os.path.join(base_dir, "labels", img_set)
    if not os.path.exists(labels_dir):
        print(f"Converting COCO → YOLO labels into {labels_dir} ...")
        os.makedirs(labels_dir, exist_ok=True)

        coco = COCO(ann_json)
        cats = coco.loadCats(coco.getCatIds())
        # COCO category ids are not 0..79; map to contiguous 0-based
        catid2cls = {c["id"]: i for i, c in enumerate(cats)}

        img_ids = coco.getImgIds()
        for img in coco.loadImgs(img_ids):
            w, h = img["width"], img["height"]
            ann_ids = coco.getAnnIds(imgIds=img["id"], iscrowd=False)
            anns = coco.loadAnns(ann_ids)

            lines = []
            for a in anns:
                x, y, bw, bh = a["bbox"]
                if bw <= 0 or bh <= 0:
                    continue
                x_c = (x + bw / 2) / w
                y_c = (y + bh / 2) / h
                bw_n = bw / w
                bh_n = bh / h
                cls = catid2cls[a["category_id"]]
                lines.append(f"{cls} {x_c:.6f} {y_c:.6f} {bw_n:.6f} {bh_n:.6f}")

            label_path = os.path.join(labels_dir, os.path.splitext(img["file_name"])[0] + ".txt")
            with open(label_path, "w") as f:
                f.write("\n".join(lines))

    print("COCO 2017 val images, annotations, and YOLO labels are ready.")


In [ ]:
base_coco_dir = f"{download_base_folder}/coco"

In [ ]:
prepare_coco_yolo(base_coco_dir)